In [1]:
import numpy as np
from sklearn.datasets import make_classification
from DMC_class import * 
import time
from sklearn.cluster import KMeans
import pandas as pd
k=50
t=30
n_samples=10000
X, y = make_classification(n_features=15,n_samples=n_samples,random_state=42,n_classes=k,n_informative=8)
XD=KMeans(n_clusters=t).fit(X).labels_
def checkarray(A,B):
    bandera=0
    for i in range(len(A)):
        if np.abs(A[i]-B[i])>0.0001:
            print(A[i],B[i])
            bandera=1
            print("error")
    if bandera==0:
        print("ok")

def checkMatriz(A,B):
    bandera=0
    for i in range(len(A)):
        for j in range(len(A[0])):
            if A[i,j]!=B[i,j]:
                bandera=1
                print("errror")
    if bandera==0:
        print("ok")
XD


C:\Users\salva\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


array([13, 15, 13, ..., 23,  7, 17])

In [2]:
y

array([26, 24, 30, ...,  4,  9, 40])

### compute_pi

In [3]:
def compute_pHatOrginal(XD: np.ndarray, y: np.ndarray, K: int, T: int):
    """
    Parameters
    ----------
    XD : ndarray of shape (n_samples,)
        Labels of profiles for each data point

    y : ndarray of shape (n_samples,)
        Labels

    K : int
        Number of classes

    T : int
        Number of profiles

    Returns
    -------
    pHat : ndarray of shape(K, n_profiles)
    """
    pHat = np.zeros((K, T))

    for k in range(K):
        Ik = np.where(y == k)[0]
        mk = len(Ik)
        for t in range(T):
            pHat[k, t] = np.sum(XD[Ik] == t) / mk
    return pHat

def compute_pHatOpti(XD: np.ndarray, y: np.ndarray, K: int, T: int):
    """
    Parameters
    ----------
    XD : ndarray of shape (n_samples,)
        Labels of profiles for each data point

    y : ndarray of shape (n_samples,)
        Labels

    K : int
        Number of classes

    T : int
        Number of profiles

    Returns
    -------
    pHat : ndarray of shape(K, n_profiles)
    """
    pHat = np.zeros((K, T))

    for k in range(K):
        Ik = np.where(y == k)[0]
        mk = len(Ik)
        
        pHat[k] = np.bincount(XD[Ik], minlength=T)/mk
    return pHat

start=time.time()
B=compute_pHatOrginal(XD,y,k,t)
print(time.time()-start)
start=time.time()
A=compute_pHatOpti(XD,y,k,t)
print(time.time()-start)
checkMatriz(A,B)

0.008965253829956055
0.0019936561584472656
ok


In [4]:
from sklearn.metrics import confusion_matrix
y_pred=np.random.randint(0, k, size=n_samples)
L=np.ones((k, k)) - np.eye(k)

def compute_conditional_riskWenlong(y_true: np.ndarray, y_pred: np.ndarray, K: int, L: np.ndarray):
    """
    Parameters
    ----------
    y_true : ndarray of shape (n_samples,)
        Real labels

    y_pred : ndarray of shape (n_samples,)
        Predicted labels

    K : int
        Number of classes

    L : ndarray of shape (K, K)
        Loss matrix, where K is the number of unique classes

    Returns
    -------
    R : ndarray of shape (K,)
        Conditional risk

    confmat : ndarray of shape (K, K)
        Confusion matrix
    """
    # Identify all unique classes
    unique_classes = np.unique(np.concatenate((y_true, y_pred)))
    class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}

    # Initialize the confusion matrix
    confmat = np.zeros((K, K))

    # Populate the confusion matrix
    for true_class in unique_classes:
        true_class_index = class_to_index[true_class]
        Ik = np.where(y_true == true_class)[0]
        pred_classes_indices = [class_to_index[pred] for pred in y_pred[Ik]]
        for pred_index in pred_classes_indices:
            confmat[true_class_index, pred_index] += 1

    # Normalize the rows of the confusion matrix to get probabilities
    row_sums = confmat.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # Prevent division by zero for classes not in y_true
    confmat /= row_sums

    # Calculate the conditional risk for each true category
    R = np.dot(L, confmat.T).diagonal()

    return R, confmat

def compute_conditional_riskSalva(YR, Yhat, K, L): 
    '''
    Function to compute the class-conditional risks.
    Parameters
    ----------
    YR : DataFrame
        Real labels.
    Yhat : Array
        Predicted labels.
    K : int
        Number of classes.
    L : Array
        Loss Function.

    Returns
    -------
    R : Array of floats
        Conditional risks.
    confmat : Matrix
        Confusion matrix.
    '''
    Labels=[i for i in range(K)]
    confmat=confusion_matrix(np.array(YR),np.array(Yhat),normalize='true',labels=Labels)
    R=np.sum(np.multiply(L, confmat),axis=1)

   # Is only the confuns 
    
    return R, confmat

def compute_conditional_riskCyprien(YR, Yhat, K, L): 
    '''
    Parameters
    ----------
    YR : DataFrame
        Real labels.
    Yhat : Array
        Predicted labels.
    K : int
        Number of classes.
    L : Array
        Loss Function.

    Returns
    -------
    R : Array of floats
        Conditional risks.
    confmat : Matrix
        Confusion matrix.
    '''
   
    confmat = np.zeros((K, K))
    R = np.zeros((1, K))
    YR_liste= YR.values.tolist()
    for k in range(0, K):
        mk = YR_liste.count([k+1])
        if mk > 0:
            Ik = np.where(YR == (k + 1))
            for l in range(0, K):
                confmat[k,l] = sum(Yhat[Ik[0]]==l+1)/mk
        R[0,k] = L[k, :].dot(confmat[k, :]) 
    
    return R, confmat
start=time.time()
O1,O2=compute_conditional_riskCyprien(pd.DataFrame(y+1),y_pred+1,k,L)
print(time.time()-start)
start=time.time()
W1,W2=compute_conditional_riskWenlong(y,y_pred,k,L)
print(time.time()-start)
start=time.time()
S1,S2=compute_conditional_riskSalva(y,y_pred,k,L)
print(time.time()-start)
checkMatriz(S2,W2)
checkarray(S1,W1)


0.8880321979522705
0.004982709884643555
0.0019893646240234375
ok
ok


In [5]:
def compute_piStarSalva(pHat, y_train, K, L, T, N, optionPlot, Box):
    """
    Parameters
    ----------
    pHat : Array of floats
        Probability estimate of observing the features profile in each class.
    y_train : Dataframe
        Real labels of the training set.
    K : int
        Number of classes.
    L : Array
        Loss Function.
    T : int
        Number of discrete profiles.
    N : int
        Number of iterations in the projected subgradient algorithm.
    optionPlot : int {0,1}
        1 plots figure,   0: does not plot figure.
    Box : Array
        {'none', matrix} : Box-constraints on the priors.

    Returns
    -------
    piStar : Array of floats
        Least favorable priors.
    rStar : float
        Global risks.
    RStar : Array of float
        Conditional risks.
    V_iter : Array
        Values of the V function at each iteration.
    stockpi : Array
        Values of pi at each iteration.

    """
    # IF BOX-CONSTRAINT == NONE (PROJECTION ONTO THE SIMPLEX)
    if Box is None:
        pi = compute_pi(y_train, K).reshape(1, -1)
        rStar = 0
        piStar = pi
        RStar = 0

        V_iter = []
        stockpi = np.zeros((K, N))

        for n in range(1, N + 1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))

            mu_k = np.sum(L[:, np.argmin(lambd, axis=0)] * pHat, axis=1)
            R[0,:] = mu_k
            stockpi[:,n-1] = pi[0,:]
            
            r = compute_global_risk(R, pi)
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R
                # Update pi for iteration n+1
            gamma = 1 / n
            eta = np.maximum(float(1), np.linalg.norm(R))
            w = pi + (gamma / eta) * R
            pi = proj_simplex_Condat(K, w)

        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:, t])
                mu_k = mu_k + L[k, lbar] * pHat[k, t]
            R[0, k] = mu_k
            stockpi[k, n - 1] = pi[0, k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R

        if optionPlot == 1:
            graph_convergence(V_iter)

    # IF BOX-CONSTRAINT
    if Box is not None:
        pi = compute_pi(y_train, K).reshape(1, -1)
        rStar = 0
        piStar = pi
        RStar = 0

        V_iter = []
        stockpi = np.zeros((K, N))

        for n in range(1, N + 1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))
            for k in range(0, K):
                mu_k = 0
                for t in range(0, T):
                    lbar = np.argmin(lambd[:, t])
                    mu_k = mu_k + L[k, lbar] * pHat[k, t]
                R[0, k] = mu_k
                stockpi[k, n - 1] = pi[0, k]
            r = compute_global_risk(R, pi)
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R
                # Update pi for iteration n+1
            gamma = 1 / n
            eta = np.maximum(float(1), np.linalg.norm(R))
            w = pi + (gamma / eta) * R
            pi = proj_onto_U(w, Box, K)

        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:, t])
                mu_k = mu_k + L[k, lbar] * pHat[k, t]
            R[0, k] = mu_k
            stockpi[k, n - 1] = pi[0, k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R

        if optionPlot == 1:
            graph_convergence(V_iter)

    return piStar, rStar, RStar, V_iter, stockpi

In [6]:
def compute_piStarWenlong(pHat, y_train, K, L, T, N, optionPlot, Box):
    """
    Parameters
    ----------
    pHat : Array of floats
        Probability estimate of observing the features profile in each class.
    y_train : Dataframe
        Real labels of the training set.
    K : int
        Number of classes.
    L : Array
        Loss Function.
    T : int
        Number of discrete profiles.
    N : int
        Number of iterations in the projected subgradient algorithm.
    optionPlot : int {0,1}
        1 plots figure,   0: does not plot figure.
    Box : Array
        {'none', matrix} : Box-constraints on the priors.

    Returns
    -------
    piStar : Array of floats
        Least favorable priors.
    rStar : float
        Global risks.
    RStar : Array of float
        Conditional risks.
    V_iter : Array
        Values of the V function at each iteration.
    stockpi : Array
        Values of pi at each iteration.

    """
    # IF BOX-CONSTRAINT == NONE (PROJECTION ONTO THE SIMPLEX)
    if Box is None:
        pi = compute_pi(y_train, K).reshape(1, -1)
        rStar = 0
        piStar = pi
        RStar = 0

        V_iter = []
        stockpi = np.zeros((K, N))

        for n in range(1, N + 1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))
            for k in range(0, K):
                mu_k = 0
                for t in range(0, T):
                    lbar = np.argmin(lambd[:, t])
                    mu_k = mu_k + L[k, lbar] * pHat[k, t]
                R[0, k] = mu_k
                stockpi[k, n - 1] = pi[0, k]
            r = compute_global_risk(R, pi)
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R
                # Update pi for iteration n+1
            gamma = 1 / n
            eta = np.maximum(float(1), np.linalg.norm(R))
            w = pi + (gamma / eta) * R
            pi = proj_simplex_Condat(K, w)

        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:, t])
                mu_k = mu_k + L[k, lbar] * pHat[k, t]
            R[0, k] = mu_k
            stockpi[k, n - 1] = pi[0, k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R

        if optionPlot == 1:
            graph_convergence(V_iter)

    # IF BOX-CONSTRAINT
    if Box is not None:
        pi = compute_pi(y_train, K).reshape(1, -1)
        rStar = 0
        piStar = pi
        RStar = 0

        V_iter = []
        stockpi = np.zeros((K, N))

        for n in range(1, N + 1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))
            for k in range(0, K):
                mu_k = 0
                for t in range(0, T):
                    lbar = np.argmin(lambd[:, t])
                    mu_k = mu_k + L[k, lbar] * pHat[k, t]
                R[0, k] = mu_k
                stockpi[k, n - 1] = pi[0, k]
            r = compute_global_risk(R, pi)
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R
                # Update pi for iteration n+1
            gamma = 1 / n
            eta = np.maximum(float(1), np.linalg.norm(R))
            w = pi + (gamma / eta) * R
            pi = proj_onto_U(w, Box, K)

        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:, t])
                mu_k = mu_k + L[k, lbar] * pHat[k, t]
            R[0, k] = mu_k
            stockpi[k, n - 1] = pi[0, k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R

        if optionPlot == 1:
            graph_convergence(V_iter)

    return piStar, rStar, RStar, V_iter, stockpi

In [7]:
Phat=compute_pHat(XD, y, k, t)
Phat

array([[0.04975124, 0.09950249, 0.039801  , ..., 0.02487562, 0.00497512,
        0.04975124],
       [0.00502513, 0.15075377, 0.15075377, ..., 0.03517588, 0.04020101,
        0.04020101],
       [0.        , 0.005     , 0.005     , ..., 0.06      , 0.055     ,
        0.01      ],
       ...,
       [0.01507538, 0.00502513, 0.08542714, ..., 0.01507538, 0.        ,
        0.03517588],
       [0.00502513, 0.0201005 , 0.01005025, ..., 0.0201005 , 0.01005025,
        0.06030151],
       [0.        , 0.        , 0.        , ..., 0.03535354, 0.08080808,
        0.01010101]])

In [9]:
N=10000
start=time.time()
#W1,W2,W3,W4,W5=compute_piStarWenlong(Phat, y, k, L, t, N, 0, None)
print(time.time()-start)
start=time.time()
pi,S2,S3,S4,S5=compute_piStarSalva(Phat, y, k, L, t, N, 0, None)
print(time.time()-start)


0.0
0.8850407600402832


In [58]:
checkarray(W1[0],S1[0])
print(W2-S2)
checkarray(W3[0],S3[0])



ok
-1.1102230246251565e-16
ok


In [11]:
def SoftminDMC_compute_CondRisksO(K, L, pHat, pi, lambd):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    
    Returns
    -------
    R_softminDMC : Array
        Values of the class-conditional risks at the point pi.
        
    '''
    
    T = np.shape(pHat)[1]
    F = np.zeros((K,T))
    NUM_SIGM = np.zeros((K,T))
    for t in range(0, T):
        for l in range(0, K):
            for j in range(0, K):
                F[l,t] = F[l,t] + L[j,l] * pHat[j,t] * pi[0,j]
            NUM_SIGM[l,t] = np.exp(-lambd * F[l,t])
    DENUM_SIGM = np.sum(NUM_SIGM, axis=0)

    R_softminDMC = np.zeros((1, K))
    for k in range(0, K):
        mu_k = 0
        for t in range(0, T):
            for l in range(0, K):
                sigma_l = NUM_SIGM[l,t]/DENUM_SIGM[t]
                mu_k = mu_k + L[k,l] * pHat[k,t] * sigma_l
        R_softminDMC[0,k] = mu_k

    return R_softminDMC[0]


def SoftminDMC_compute_CondRisksS(K, L, pHat, pi, lambd):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    
    Returns
    -------
    R_softminDMC : Array
        Values of the class-conditional risks at the point pi.
    '''
    
    T = pHat.shape[1]
    
    # Compute F using vectorized operations
    F = np.einsum('jl,jt,j->lt', L, pHat, pi[0])
    
    # Compute NUM_SIGM and DENUM_SIGM using vectorized operations
    NUM_SIGM = np.exp(-lambd * F)
    DENUM_SIGM = np.sum(NUM_SIGM, axis=0)
    
    # Compute sigma_l
    sigma = NUM_SIGM / DENUM_SIGM
    
    # Compute mu_k and R_softminDMC using vectorized operations
    R_softminDMC = np.einsum('kl,kt,lt->k', L, pHat, sigma)
    
    return R_softminDMC

In [16]:
# Ejemplo de uso:

start=time.time()
resultado1 = SoftminDMC_compute_CondRisksO(k, L, Phat, pi, 2)
print(time.time()-start)
start=time.time()
resultado2 = SoftminDMC_compute_CondRisksS(k, L, Phat, pi, 2)
print(time.time()-start)

#print(resultado1)
#print(resultado2)
checkarray(resultado1,resultado2)

0.09667825698852539
0.0009937286376953125
ok


In [8]:
k

50